In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv("../../../data/raw/vaastav_2022_23.csv") 

In [3]:
df.head()

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,starts,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,Nathan Redmond,MID,Southampton,1.5,0,0,3,0,0.0,403,0.0,0.0,0.0,0.0,8,0,0,0.0,0.0,2022-08-06T14:00:00Z,1,18,0,0,0,0,1,0,5871,0,1,4,0.0,1,0,0,0,55,False,0,1
1,Junior Stanislas,MID,Bournemouth,1.1,0,0,3,0,0.0,58,0.0,0.0,0.0,0.0,2,0,0,0.0,0.0,2022-08-06T14:00:00Z,1,2,0,0,0,0,1,0,9491,0,0,2,0.0,1,0,0,0,50,True,0,1
2,Armando Broja,FWD,Chelsea,2.0,0,0,3,0,0.3,150,0.0,0.0,0.0,0.0,3,0,0,2.5,5.2,2022-08-06T16:30:00Z,15,8,0,0,0,0,1,0,124903,0,1,0,19.0,1,0,0,0,55,False,0,1
3,Fabian Schär,DEF,Newcastle,2.4,0,3,43,1,14.6,366,0.0,0.0,0.0,0.0,7,0,1,10.6,66.0,2022-08-06T14:00:00Z,90,16,0,0,0,0,1,0,40132,0,0,2,25.0,15,0,0,0,45,True,0,1
4,Jonny Evans,DEF,Leicester,1.9,0,0,15,0,1.3,249,0.0,0.0,0.0,0.0,6,2,0,1.5,14.0,2022-08-07T13:00:00Z,90,4,0,0,0,0,1,0,24256,0,2,2,0.0,1,0,0,0,45,True,0,1


In [4]:
df.columns

Index(['name', 'position', 'team', 'xP', 'assists', 'bonus', 'bps',
       'clean_sheets', 'creativity', 'element', 'expected_assists',
       'expected_goal_involvements', 'expected_goals',
       'expected_goals_conceded', 'fixture', 'goals_conceded', 'goals_scored',
       'ict_index', 'influence', 'kickoff_time', 'minutes', 'opponent_team',
       'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards',
       'round', 'saves', 'selected', 'starts', 'team_a_score', 'team_h_score',
       'threat', 'total_points', 'transfers_balance', 'transfers_in',
       'transfers_out', 'value', 'was_home', 'yellow_cards', 'GW'],
      dtype='object')

In [5]:
all(df['round'] == df['GW'])

True

In [6]:
cols_to_drop = ['fixture', 'kickoff_time', 'selected', 'transfers_balance', 'transfers_in', 'transfers_out', 'round']

In [7]:
trimmed_df = df.drop(columns=cols_to_drop)

In [8]:
trimmed_df.head()

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,goals_conceded,goals_scored,ict_index,influence,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,saves,starts,team_a_score,team_h_score,threat,total_points,value,was_home,yellow_cards,GW
0,Nathan Redmond,MID,Southampton,1.5,0,0,3,0,0.0,403,0.0,0.0,0.0,0.0,0,0,0.0,0.0,1,18,0,0,0,0,0,0,1,4,0.0,1,55,False,0,1
1,Junior Stanislas,MID,Bournemouth,1.1,0,0,3,0,0.0,58,0.0,0.0,0.0,0.0,0,0,0.0,0.0,1,2,0,0,0,0,0,0,0,2,0.0,1,50,True,0,1
2,Armando Broja,FWD,Chelsea,2.0,0,0,3,0,0.3,150,0.0,0.0,0.0,0.0,0,0,2.5,5.2,15,8,0,0,0,0,0,0,1,0,19.0,1,55,False,0,1
3,Fabian Schär,DEF,Newcastle,2.4,0,3,43,1,14.6,366,0.0,0.0,0.0,0.0,0,1,10.6,66.0,90,16,0,0,0,0,0,0,0,2,25.0,15,45,True,0,1
4,Jonny Evans,DEF,Leicester,1.9,0,0,15,0,1.3,249,0.0,0.0,0.0,0.0,2,0,1.5,14.0,90,4,0,0,0,0,0,0,2,2,0.0,1,45,True,0,1


In [9]:
trimmed_df.loc[trimmed_df['element'] == 30]

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,goals_conceded,goals_scored,ict_index,influence,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,saves,starts,team_a_score,team_h_score,threat,total_points,value,was_home,yellow_cards,GW
526,Danny Ings,FWD,Aston Villa,2.7,0,0,0,0,12.5,30,0.00000,0.00000,0.0000,0.0000,1,0,4.7,1.0,65,3,0,0,0,0,0,0,0,2,33.0,1,70,False,1,1
1105,Danny Ings,FWD,Aston Villa,3.5,0,1,23,1,10.0,30,0.00000,0.00000,0.0000,0.0000,0,1,8.9,31.8,77,8,0,0,0,0,0,0,1,2,47.0,6,70,True,1,2
1696,Danny Ings,FWD,Aston Villa,2.7,0,0,1,0,0.0,30,0.00000,0.00000,0.0000,0.0000,1,0,1.8,1.0,24,7,0,0,0,0,0,0,1,3,17.0,1,70,False,0,3
2297,Danny Ings,FWD,Aston Villa,2.5,0,0,2,0,1.8,30,0.00000,0.00000,0.0000,0.0000,1,0,0.6,0.0,77,19,0,0,0,0,0,0,1,0,6.0,2,69,True,0,4
2903,Danny Ings,FWD,Aston Villa,1.7,0,0,3,0,0.1,30,0.00000,0.00000,0.0000,0.0000,0,0,0.2,0.2,7,1,0,0,0,0,0,0,1,2,2.0,1,69,False,0,5
3526,Danny Ings,FWD,Aston Villa,1.0,0,0,0,0,0.0,30,0.00000,0.00000,0.0000,0.0000,0,0,0.0,0.0,0,13,0,0,0,0,0,0,1,1,0.0,0,68,True,0,6
3981,Danny Ings,FWD,Aston Villa,1.0,0,0,-2,0,1.6,30,0.00000,0.00000,0.0000,0.0000,0,0,0.1,0.0,6,17,0,0,0,0,0,0,0,1,2.0,1,67,True,0,8
4604,Danny Ings,FWD,Aston Villa,1.0,0,0,4,0,10.3,30,0.00000,0.00000,0.0000,0.0000,0,0,1.2,2.0,7,11,0,0,0,0,0,0,0,0,0.0,1,67,False,0,9
5245,Danny Ings,FWD,Aston Villa,1.5,0,0,2,0,0.5,30,0.00000,0.00000,0.0000,0.0000,0,0,0.3,2.0,25,16,0,0,0,0,0,0,1,1,0.0,1,67,False,0,10
5888,Danny Ings,FWD,Aston Villa,0.8,0,0,0,0,0.8,30,0.00000,0.00000,0.0000,0.0000,2,0,9.6,11.0,90,6,0,0,0,0,0,0,2,0,84.0,2,67,True,0,11


In [10]:
df.columns

Index(['name', 'position', 'team', 'xP', 'assists', 'bonus', 'bps',
       'clean_sheets', 'creativity', 'element', 'expected_assists',
       'expected_goal_involvements', 'expected_goals',
       'expected_goals_conceded', 'fixture', 'goals_conceded', 'goals_scored',
       'ict_index', 'influence', 'kickoff_time', 'minutes', 'opponent_team',
       'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards',
       'round', 'saves', 'selected', 'starts', 'team_a_score', 'team_h_score',
       'threat', 'total_points', 'transfers_balance', 'transfers_in',
       'transfers_out', 'value', 'was_home', 'yellow_cards', 'GW'],
      dtype='object')

In [11]:
cols_to_keep = ['element', 'position', 'goals_scored', 'assists', 'bonus', 'bps', 'expected_goals', 'expected_assists', 'total_points', 'GW', ]

In [12]:
df1 = df[cols_to_keep]

In [13]:
df2 = df.pivot_table(index='element', 
                columns='GW', 
                values=['goals_scored', 'assists', 'bonus', 'bps', 'expected_goals', 'expected_assists', 'total_points'], 
                aggfunc='sum').reset_index()

df2.columns = [f"{col}_gw{int(gw)}" if isinstance(gw, (int,float)) else col for col,gw in df2.columns]

In [14]:
df2.shape

(778, 260)

In [15]:
df.columns

Index(['name', 'position', 'team', 'xP', 'assists', 'bonus', 'bps',
       'clean_sheets', 'creativity', 'element', 'expected_assists',
       'expected_goal_involvements', 'expected_goals',
       'expected_goals_conceded', 'fixture', 'goals_conceded', 'goals_scored',
       'ict_index', 'influence', 'kickoff_time', 'minutes', 'opponent_team',
       'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards',
       'round', 'saves', 'selected', 'starts', 'team_a_score', 'team_h_score',
       'threat', 'total_points', 'transfers_balance', 'transfers_in',
       'transfers_out', 'value', 'was_home', 'yellow_cards', 'GW'],
      dtype='object')

In [16]:
# select features
cols_to_keep = ['minutes',
                'team',
                'goals_scored', 
                'assists', 
                'expected_goals', 
                'expected_assists', 
                'clean_sheets',
                'ict_index',
                'bps', 
                'bonus', 
                'total_points',
                ]

# pivot table to get unified dataframe
df_multigw = df.pivot_table(index='element', 
                columns = 'GW', 
                values = cols_to_keep, 
                aggfunc='sum').reset_index()

# rename columns to include gameweek number
df_multigw.columns = [f"{col}_gw{int(gw)}" if isinstance(gw, (int,float)) else col for col,gw in df_multigw.columns]

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26505 entries, 0 to 26504
Data columns (total 41 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   name                        26505 non-null  object 
 1   position                    26505 non-null  object 
 2   team                        26505 non-null  object 
 3   xP                          26505 non-null  float64
 4   assists                     26505 non-null  int64  
 5   bonus                       26505 non-null  int64  
 6   bps                         26505 non-null  int64  
 7   clean_sheets                26505 non-null  int64  
 8   creativity                  26505 non-null  float64
 9   element                     26505 non-null  int64  
 10  expected_assists            26505 non-null  float64
 11  expected_goal_involvements  26505 non-null  float64
 12  expected_goals              26505 non-null  float64
 13  expected_goals_conceded     265

In [22]:
df.groupby('element').agg({'team': 'first',
                           'position': 'first'
                           }).reset_index()

,element,team,position
0,1,Arsenal,DEF
1,2,Fulham,GK
2,3,Arsenal,MID
3,4,Arsenal,MID
4,5,Arsenal,DEF
5,6,Arsenal,MID
6,7,Arsenal,MID
7,8,Arsenal,DEF
8,9,Arsenal,MID
9,10,Arsenal,DEF
